# Explain Model – Interactive Manual

This notebook is a hands-on manual for the full Explain model runtime. It helps you:

- load any definition from `definitions/`
- inspect instantiated models and their attributes
- step the simulation interactively
- collect and plot numeric signals

Run cells from top to bottom once, then use the interactive panel.

## Runtime architecture (quick reference)

1. `ModelEngine` loads a definition (`general`, `models`, `components`, `helpers`).
2. Each entry resolves by `model_type` to a `BaseModel` subclass.
3. `init_model(...)` applies config and can build nested components.
4. `step_model()` advances all enabled models one simulation step.

Core files:
- `model_engine.py`
- `base_models/base_model.py`
- `docs/model_structure.md`

In [6]:
from pathlib import Path
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

REPO_ROOT = Path.cwd()
if not (REPO_ROOT / 'model_engine.py').exists():
    # fallback when notebook is opened from another working directory
    for parent in [Path.cwd(), *Path.cwd().parents]:
        if (parent / 'model_engine.py').exists():
            REPO_ROOT = parent
            break

if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from model_engine import ModelEngine

DEFINITIONS_DIR = REPO_ROOT / 'definitions'
DEFINITIONS = sorted([p.name for p in DEFINITIONS_DIR.glob('*.json')])

print(f'Repository root: {REPO_ROOT}')
print(f'Found {len(DEFINITIONS)} definitions')
DEFINITIONS[:10]

Repository root: /Users/timantonius/Projects/explain/explain-python
Found 1 definitions


['baseline_neonate.json']

In [2]:
def load_engine(definition_name: str) -> ModelEngine:
    engine = ModelEngine()
    engine.load_json_file(str(DEFINITIONS_DIR / definition_name))
    return engine


def model_type_counts(engine: ModelEngine):
    counts = {}
    for name, model in engine.models.items():
        key = getattr(model, 'model_type', type(model).__name__)
        counts[key] = counts.get(key, 0) + 1
    return dict(sorted(counts.items(), key=lambda x: (-x[1], x[0])))


def numeric_attrs(obj):
    result = []
    for attr in dir(obj):
        if attr.startswith('_'):
            continue
        try:
            value = getattr(obj, attr)
        except Exception:
            continue
        if isinstance(value, (int, float)) and math.isfinite(float(value)):
            result.append(attr)
    return sorted(result)


def safe_get(engine: ModelEngine, model_name: str, attr: str):
    model = engine.models.get(model_name)
    if model is None or not hasattr(model, attr):
        return np.nan
    value = getattr(model, attr)
    if isinstance(value, (int, float)):
        v = float(value)
        return v if math.isfinite(v) else np.nan
    return np.nan

## Interactive control panel

In [ ]:
state = {
    'engine': None,
    'definition': None,
}

definition_dd = widgets.Dropdown(
    options=DEFINITIONS,
    value='baseline_mongo.json' if 'baseline_mongo.json' in DEFINITIONS else (DEFINITIONS[0] if DEFINITIONS else None),
    description='Definition:',
    layout=widgets.Layout(width='420px')
)

load_btn = widgets.Button(description='Load definition', button_style='primary')
step_count = widgets.BoundedIntText(value=100, min=1, max=500000, description='Steps:')
run_btn = widgets.Button(description='Run steps', button_style='success')

model_dd = widgets.Dropdown(options=[], description='Model:', layout=widgets.Layout(width='420px'))
max_attrs = widgets.BoundedIntText(value=30, min=1, max=500, description='Show attrs:')
inspect_btn = widgets.Button(description='Inspect model')

signal_model_dd = widgets.Dropdown(options=[], description='Signal model:', layout=widgets.Layout(width='420px'))
signal_attr_dd = widgets.Dropdown(options=[], description='Signal attr:', layout=widgets.Layout(width='420px'))
plot_steps = widgets.BoundedIntText(value=2000, min=10, max=1000000, description='Plot steps:')
plot_btn = widgets.Button(description='Collect + plot', button_style='warning')

status_out = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='8px'))
summary_out = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='8px'))
inspect_out = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='8px'))
plot_out = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='8px'))


def refresh_model_selectors():
    engine = state['engine']
    if engine is None:
        model_dd.options = []
        signal_model_dd.options = []
        signal_attr_dd.options = []
        return

    names = sorted(engine.models.keys())
    model_dd.options = names
    signal_model_dd.options = names

    preferred = 'Monitor' if 'Monitor' in engine.models else (names[0] if names else None)
    if preferred is not None:
        model_dd.value = preferred
        signal_model_dd.value = preferred
        on_signal_model_change(None)


def write_summary():
    summary_out.clear_output()
    engine = state['engine']
    with summary_out:
        if engine is None:
            print('No model loaded.')
            return

        print(f'Initialized: {engine.is_initialized}')
        print(f'Model count: {len(engine.models)}')
        print(f'Modeling step size: {getattr(engine, "modeling_stepsize", None)} s')

        print('\nTop model_type frequencies:')
        for i, (model_type, count) in enumerate(model_type_counts(engine).items()):
            if i >= 15:
                print('  ...')
                break
            print(f'  {model_type}: {count}')


def on_load_clicked(_):
    status_out.clear_output()
    with status_out:
        try:
            selected = definition_dd.value
            if selected is None:
                print('No definition file found in definitions/.')
                return
            engine = load_engine(selected)
            state['engine'] = engine
            state['definition'] = selected
            print(f'Loaded: {selected}')
            print(f'Models: {len(engine.models)}')
        except Exception as exc:
            state['engine'] = None
            print(f'Load failed: {exc}')

    refresh_model_selectors()
    write_summary()


def on_run_clicked(_):
    engine = state['engine']
    status_out.clear_output()
    with status_out:
        if engine is None:
            print('Load a definition first.')
            return

        n = int(step_count.value)
        for _i in range(n):
            engine.step_model()

        print(f'Ran {n} steps on {state["definition"]}.')

        monitor = engine.models.get('Monitor') or engine.models.get('MON')
        if monitor is not None:
            hr = getattr(monitor, 'heart_rate', None)
            map_ = getattr(monitor, 'abp_mean', None)
            spo2 = getattr(monitor, 'spo2', getattr(monitor, 'spo2_pre', None))
            print(f'Monitor snapshot: heart_rate={hr}, abp_mean={map_}, spo2={spo2}')

    write_summary()


def on_inspect_clicked(_):
    inspect_out.clear_output()
    engine = state['engine']
    with inspect_out:
        if engine is None:
            print('Load a definition first.')
            return

        name = model_dd.value
        model = engine.models.get(name)
        if model is None:
            print('Model not found.')
            return

        print(f'Model: {name}')
        print(f'Class: {model.__class__.__name__}')
        print(f'model_type: {getattr(model, "model_type", None)}')
        print(f'is_enabled: {getattr(model, "is_enabled", None)}')

        attrs = []
        for attr in dir(model):
            if attr.startswith('_'):
                continue
            try:
                val = getattr(model, attr)
            except Exception:
                continue
            if callable(val):
                continue
            attrs.append((attr, val))

        limit = int(max_attrs.value)
        print(f'\nShowing up to {limit} attributes:')
        for attr, val in attrs[:limit]:
            txt = repr(val)
            if len(txt) > 120:
                txt = txt[:117] + '...'
            print(f'  {attr}: {txt}')


def on_signal_model_change(_):
    engine = state['engine']
    if engine is None:
        signal_attr_dd.options = []
        return

    model_name = signal_model_dd.value
    model = engine.models.get(model_name)
    if model is None:
        signal_attr_dd.options = []
        return

    attrs = numeric_attrs(model)
    signal_attr_dd.options = attrs

    preferred_attrs = ['heart_rate', 'abp_mean', 'spo2', 'resp_rate', 'ecg_signal', 'pres', 'flow']
    for pref in preferred_attrs:
        if pref in attrs:
            signal_attr_dd.value = pref
            break


def on_plot_clicked(_):
    plot_out.clear_output()
    engine = state['engine']
    with plot_out:
        if engine is None:
            print('Load a definition first.')
            return

        model_name = signal_model_dd.value
        attr_name = signal_attr_dd.value
        if not model_name or not attr_name:
            print('Select a signal model and attribute.')
            return

        n = int(plot_steps.value)
        x = np.arange(n)
        y = np.zeros(n, dtype=float)

        for i in range(n):
            engine.step_model()
            y[i] = safe_get(engine, model_name, attr_name)

        valid = np.isfinite(y)
        if valid.any():
            y_min = float(np.nanmin(y[valid]))
            y_max = float(np.nanmax(y[valid]))
            y_last = float(y[valid][-1])
        else:
            y_min = y_max = y_last = float('nan')

        plt.figure(figsize=(10, 4))
        plt.plot(x, y, lw=1.25)
        plt.title(f'{model_name}.{attr_name} over {n} steps')
        plt.xlabel('Step')
        plt.ylabel(attr_name)
        plt.grid(alpha=0.25)
        plt.show()

        print(f'Collected {n} steps from {model_name}.{attr_name}')
        print(f'last={y_last:.6g}, min={y_min:.6g}, max={y_max:.6g}')


load_btn.on_click(on_load_clicked)
run_btn.on_click(on_run_clicked)
inspect_btn.on_click(on_inspect_clicked)
plot_btn.on_click(on_plot_clicked)
signal_model_dd.observe(on_signal_model_change, names='value')

panel = widgets.VBox([
    widgets.HBox([definition_dd, load_btn]),
    widgets.HBox([step_count, run_btn]),
    widgets.HTML('<b>Model inspection</b>'),
    widgets.HBox([model_dd, max_attrs, inspect_btn]),
    widgets.HTML('<b>Signal plotting</b>'),
    widgets.HBox([signal_model_dd, signal_attr_dd]),
    widgets.HBox([plot_steps, plot_btn]),
    widgets.HTML('<b>Status</b>'),
    status_out,
    widgets.HTML('<b>Engine summary</b>'),
    summary_out,
    widgets.HTML('<b>Inspection output</b>'),
    inspect_out,
    widgets.HTML('<b>Plot output</b>'),
    plot_out,
])

display(panel)

# Auto-load default definition once for convenience
if definition_dd.value is not None:
    on_load_clicked(None)

## Suggested learning workflow

1. Load `baseline_mongo.json` and inspect `Monitor`, `Heart`, `Breathing`, `Circulation`.
2. Run 100–1000 steps and watch monitor snapshot updates.
3. Plot `Monitor.heart_rate`, `Monitor.abp_mean`, `Monitor.spo2`, and `Heart.ecg_signal`.
4. Switch definitions (e.g. `ecls.json`, `ventilator.json`, `resuscitation.json`) and compare model counts and signals.
5. Inspect subsystem classes to connect runtime behavior to model implementations.

## Guided chapters (tab view)

Use this section as a structured manual for major subsystems. Click **Refresh chapters** after loading a definition or running steps.

In [4]:
# Chapter tabs: Cardiovascular, Respiratory, Devices, ANS

chapter_out = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='8px'))
refresh_chapters_btn = widgets.Button(description='Refresh chapters', button_style='info')


def _is_device(model_name: str, model):
    n = model_name.upper()
    cls = model.__class__.__name__.lower()
    mtype = str(getattr(model, 'model_type', '')).lower()
    device_tokens = ['vent', 'ecls', 'resus', 'monitor', 'pump']
    return any(tok in n.lower() or tok in cls or tok in mtype for tok in device_tokens)


def _is_respiratory(model_name: str, model):
    n = model_name.upper()
    cls = model.__class__.__name__.lower()
    mtype = str(getattr(model, 'model_type', '')).lower()
    tokens = ['resp', 'breath', 'lung', 'air', 'gas', 'alv', 'ettube']
    return any(tok in n.lower() or tok in cls or tok in mtype for tok in tokens)


def _is_ans(model_name: str, model):
    n = model_name.upper()
    cls = model.__class__.__name__.lower()
    mtype = str(getattr(model, 'model_type', '')).lower()
    return ('ANS' in n) or ('ans' in cls) or ('ans' in mtype)


def _cardio_default(model_name: str, model):
    return not (_is_device(model_name, model) or _is_respiratory(model_name, model) or _is_ans(model_name, model))


def _chapter_lines(engine: ModelEngine, chapter: str):
    items = []

    for name, model in engine.models.items():
        if chapter == 'Cardiovascular' and _cardio_default(name, model):
            items.append((name, model))
        elif chapter == 'Respiratory' and _is_respiratory(name, model):
            items.append((name, model))
        elif chapter == 'Devices' and _is_device(name, model):
            items.append((name, model))
        elif chapter == 'ANS' and _is_ans(name, model):
            items.append((name, model))

    items.sort(key=lambda x: x[0])
    lines = []
    lines.append(f'Chapter: {chapter}')
    lines.append(f'Models in chapter: {len(items)}')
    lines.append('')

    preview = items[:40]
    for name, model in preview:
        model_type = getattr(model, 'model_type', model.__class__.__name__)
        enabled = getattr(model, 'is_enabled', None)
        lines.append(f'- {name} ({model_type}), enabled={enabled}')

    if len(items) > len(preview):
        lines.append(f'... and {len(items) - len(preview)} more')

    lines.append('')
    lines.append('Suggested signals to inspect:')
    if chapter == 'Cardiovascular':
        lines.append('- Monitor.heart_rate, Monitor.abp_mean, Heart.ecg_signal, AA.pres')
    elif chapter == 'Respiratory':
        lines.append('- Monitor.resp_rate, DS.pres, Breathing.resp_muscle_pressure')
    elif chapter == 'Devices':
        lines.append('- VENT.pres/flow, ECLS.blood_flow, Monitor.etco2')
    elif chapter == 'ANS':
        lines.append('- ANS activity flags, ANS_EFF firing fields, target factor changes')

    return '\n'.join(lines)


def refresh_chapters(_=None):
    chapter_out.clear_output()
    with chapter_out:
        engine = state.get('engine') if 'state' in globals() else None
        if engine is None:
            print('Load a definition in the interactive control panel first.')
            return

        tabs = widgets.Tab()
        outputs = []
        chapter_names = ['Cardiovascular', 'Respiratory', 'Devices', 'ANS']

        for chapter in chapter_names:
            out = widgets.Output()
            with out:
                print(_chapter_lines(engine, chapter))
            outputs.append(out)

        tabs.children = outputs
        for i, chapter in enumerate(chapter_names):
            tabs.set_title(i, chapter)

        display(tabs)


refresh_chapters_btn.on_click(refresh_chapters)
display(widgets.VBox([refresh_chapters_btn, chapter_out]))

# Render once immediately
refresh_chapters()